In [ ]:
import numpy as np
import os
import time
import scipy
from collections import defaultdict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# making dictionaries to count frequency of features.
api_calls_count = defaultdict(int)
permissions_count = defaultdict(int)
intents_count = defaultdict(int)
strings_count = defaultdict(int)

# read an app's features (API calls, Permissions and constant strings) from stored files.
def read_individual_app_features(app_path):
    '''
    Input:
        File's path to extract features from.
    Return:
        List containg features name in string form.
    '''
    
    app_feature = []
    try:
        app_file = open(app_path,'r') # Open the file containg app's features
        for feature in app_file:
            app_feature.append(feature.rstrip()) # append every line(feature) in a list
    except:
        pass
    
    return list(set(app_feature))

# reading App intents (we read intents in different way because intents are within APP components file.)
def reading_intents(app_path):
    intents = []
    file = open(app_path, 'r')
    while(file.readline()):
        if(file.readline().rstrip() == 'Intents:'):
            while(file.readline()):
                intents.append(file.readline().rstrip())
    return list(set(intents))

# reading apps from directory one by one to read their properties.
def reading_features(directory_path, sha):
    
    apk_features = []
    
    #extract permissions from stored apk's permissions file
    permissions = read_individual_app_features(os.path.join(directory_path + 'Test_APKs_Permissions/' + sha)) 
    for perm in permissions:
        permissions_count[perm] += 1
    apk_features += permissions
    
    #extract api_calls from stored apk's api_calls file
    api_calls = read_individual_app_features(os.path.join(directory_path + 'Test_API_calls/' + sha))
    for api_call in api_calls:
        api_calls_count[api_call] += 1
    apk_features += api_calls
    
    #extract intents from stored apk's intents file.
    intents = reading_intents(os.path.join(directory_path + 'Test_APKs_App_Components/' + sha))
    for intent in intents:
        intents_count[intent] += 1
    apk_features += intents
    
    #extract constant_strings from stored apk's constant strings file.
    constant_strings = read_individual_app_features(os.path.join(directory_path + 'Test_Constant_Strings/' + sha))
    for string in constant_strings:
        strings_count[string] += 1
    apk_features += constant_strings
    
    return apk_features

In [ ]:
# Feature's name from Malware dataset
start_time = time.time()

malware_unique_features = [] # stroing a features in a string form as a list element

malware_directory_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/APKs_Properties_Files/'
malware_apks_shas = os.listdir('/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/APKs_Properties_Files/APKs_App_Components/')

count = 0
for sha in malware_apks_shas:
    malware_unique_features += reading_features(malware_directory_path, sha) # add benign directry app's features to list
    count+=1
    if(count%500 == 0):
        print(count)
        
malware_unique_features = list(set(malware_unique_features))
print("Time taken by extracting features from benign apps is ", (time.time()-start_time)/60.0, ' minutes.' )
print(f'lenight of malware uique features is : {len(malware_unique_features)}')


# Feature's name from Benign dataset
start_time = time.time()

benign_unique_features = []
benign_directory_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Androguard+Predefined_Dictionary/APKs_Properties_Files/'
benign_apks_shas = os.listdir('/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Androguard+Predefined_Dictionary/APKs_Properties_Files/APKs_App_Components/')

count = 0
for sha in benign_apks_shas:
    benign_unique_features += reading_features(benign_directory_path, sha) # add benign directry app's features to list
    count+=1
    if(count%500 == 0):
        print(count)

benign_unique_features = list(set(benign_unique_features))
print("Time taken by extracting features from benign apps is ", (time.time()-start_time)/60.0, ' minutes.' )
print(f'length of unique benign featres is: {len(benign_unique_features)}')

unique_features = list(set(benign_unique_features + malware_unique_features))

print(f'Total unique features of both bening and malware are : {len(unique_features)}')

FileNotFoundError: ignored

In [ ]:
#Sorted the features based on frequency in descending order.
'''api_calls_count = {k: v for k, v in sorted(api_calls_count.items(), key=lambda item: item[1], reverse=True)}
permissions_count = {k: v for k, v in sorted(permissions_count.items(), key=lambda item: item[1], reverse=True)}
intents_count = {k: v for k, v in sorted(intents_count.items(), key=lambda item: item[1], reverse=True)}
strings_count = {k: v for k, v in sorted(strings_count.items(), key=lambda item: item[1], reverse=True)}'''

'api_calls_count = {k: v for k, v in sorted(api_calls_count.items(), key=lambda item: item[1], reverse=True)}\npermissions_count = {k: v for k, v in sorted(permissions_count.items(), key=lambda item: item[1], reverse=True)}\nintents_count = {k: v for k, v in sorted(intents_count.items(), key=lambda item: item[1], reverse=True)}\nstrings_count = {k: v for k, v in sorted(strings_count.items(), key=lambda item: item[1], reverse=True)}'

Since we have to create binary image of shape, 256x256 to represent an APK. So we want to consider first 65536 (256*256) frequent called features.

In [ ]:
print(len(api_calls_count))
print(len(permissions_count))
print(len(intents_count))
print(len(strings_count))

138728
712
2702
18362


In [ ]:
top_api_calls = []
for api_call, frequency in api_calls_count.items():
    if(frequency>=2):
        top_api_calls.append(api_call)
print(f'There are {len(top_api_calls)} api calls, called by at least 2 apks.')

top_permissions = []
for permission, frequency in permissions_count.items():
    if(frequency>=2):
        top_permissions.append(permission)
print(f'There are {len(top_permissions)} permissions, called by at least 2 apks.')

top_cnstnt_strngs = []
for string, frequency in strings_count.items():
    if(frequency>=3):
        top_cnstnt_strngs.append(string)
print(f'There are {len(top_cnstnt_strngs)} constant strings, called by at least 3 apks.')

top_intents = []
for intent, frequency in intents_count.items():
    if(frequency>=2):
        top_intents.append(intent)
print(f'There are {len(top_intents)} intents, called by 2 apks.')

There are 60579 api calls, called by at least 2 apks.
There are 402 permissions, called by at least 2 apks.
There are 3790 constant strings, called by at least 3 apks.
There are 619 intents, called by 2 apks.


In [ ]:
unique_features = list(api_calls_count.keys())[:60500] + list(permissions_count.keys())[:500] + list(strings_count.keys())[:3936] + list(intents_count.keys())[:600]
print(len(unique_features))

65536


In [ ]:
with open('/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/APKs_Properties_Files/Unique_features', 'w') as file:
    for feature in unique_features:
        file.write("%s\n" %feature)

In [ ]:
unique_features = []
for feature in open('/content/drive/My Drive/Research-Adverserial_ML/P9:Malware_Detection_Based_on_CNN/Datasets/Unique_features', 'r'):
  unique_features.append(feature.rstrip())
print(len(unique_features))

65536


In [ ]:
start_time = time.time()

malware_matrix = [] # binary matrix of whole malware dataset

malware_directory_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/APKs_Properties_Files/'
malware_apks_shas = os.listdir('/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/APKs_Properties_Files/APKs_App_Components/')
shas_100_list = []
count = 0
for sha in malware_apks_shas:
    shas_100_list.append(sha)
    apk_vector = np.zeros(65536, dtype=np.int16) # vectors representing an APK
    apk_features = reading_features(malware_directory_path, sha) # add benign directry app's features to list
    
    apk_indices = []
    for feature in apk_features:
        try:
            apk_indices.append(unique_features.index(feature))
        except:
            continue
    apk_vector[apk_indices] = 1
    apk_vector = apk_vector.reshape(256,256)
    #apk_indices = [unique_features.index(feature) for feature in apk_features] # 
    
    malware_matrix.append(apk_vector)
    count+=1
    if(count%500 == 0):
        print(count)

malware_matrix = np.array(malware_matrix)
print(malware_matrix.shape)

print("Time taken by extracting features from malware apps is ", (time.time()-start_time)/60.0, ' minutes.' )

FileNotFoundError: ignored

In [ ]:
test_normal_100 = benign_matrix[:1000]

In [ ]:
np.save('/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/Inter_Linear/11k_100_AEs_Testing/AEs_for_testing_200/100_same_APKs_properties_Files/Normal_1000_same_benign.npy', test_normal_100)

In [ ]:
np.save('/home/mujeeb/Adverserial_ML/P9:Malware_Detection_Based_on_CNN/malware_matrix_updated.npy', malware_matrix)

In [ ]:
start_time = time.time()

benign_matrix = [] # binary matrix of whole malware dataset

benign_directory_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Androguard+Predefined_Dictionary/APKs_Properties_Files/'
benign_shas = os.listdir('/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Androguard+Predefined_Dictionary/APKs_Properties_Files/APKs_App_Components/')

count = 0
for sha in benign_shas:
    apk_vector = np.zeros(65536, dtype=np.int16) # vectors representing an APK
    apk_features = reading_features(benign_directory_path, sha) # add benign directry app's features to list
    
    apk_indices = []
    for feature in apk_features:
        try:
            apk_indices.append(unique_features.index(feature))
        except:
            continue
    apk_vector[apk_indices] = 1
    apk_vector = apk_vector.reshape(256,256)
    #apk_indices = [unique_features.index(feature) for feature in apk_features] # 
    
    benign_matrix.append(apk_vector)
    count+=1
    if(count%500 == 0):
        print(count)

benign_matrix = np.array(benign_matrix)
print(benign_matrix.shape)

print("Time taken by extracting features from benign apps is ", (time.time()-start_time)/60.0, ' minutes.' )

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
(5591, 256, 256)
Time taken by extracting features from benign apps is  51.89874167442322  minutes.


In [ ]:
np.save('/home/mujeeb/Adverserial_ML/P9:Malware_Detection_Based_on_CNN/benign_matrix_updated.npy', benign_matrix)

## Adding perturbation

**Adding Perturbation Via Attack1**

In [ ]:
benign_permissions = {"FOREGROUND_SERVICE": 216, "UA_DATA": 82, "PushHandlerActivity": 58, "USE_BIOMETRIC": 16, "A4S_SEND": 16, "INTERNAL_BROADCAST": 16, "BIND_NOTIFICATION_LISTENER_SERVICE": 11, "GALLERY_PROVIDER": 11, "PUSH": 10, "ACCESS_ACCOUNT": 10, "GOOGLE_PHOTOS": 10, "REQUEST_DELETE_PACKAGES": 10, "LAYER_PUSH": 10, "CONFIGURE_SIP": 9, "ACTION_HEADSET_PLUG": 9, "ACCESS_CORSE_LOCATION": 8, "REGISTER": 8, "READ_PHONE_NUMBERS": 8, "INBILLING": 8, "ACCESS_FINGERPRINT_MANAGER": 7, "PROVIDE_BACKGROUND": 7, "ANSWER_PHONE_CALLS": 7, "ACCESS_BACKGROUND_LOCATION": 6, "GI3_E97F0800__FE8AC018_D1FA_4683_98C9_02D777CD118B__INTERNAL_BROADCAST": 6, "A4S_READ_DATABASE": 6, "LOCATION_HARDWARE": 6, "READ_DATABASE": 6, "BOOKS": 6, "PROCESS_PUSH_MSG": 6, "A4S_WRITE_DATABASE": 6, "READ_CREDENTIALS": 6, "RECEIVE_MSG": 6, "NETWORK": 6, "READ_CONTENT_PROVIDER": 5, "PRIVATE": 5, "ACTION_MANAGE_OVERLAY_PERMISSION": 5, "WRITE_DATABASE": 5, "BIND_INPUT_METHOD": 5, "PERMISSION_VSIR": 4, "RECEIVE_FIRST_LOAD_BROADCAST": 4, "BROADCAST_RECEIVER": 4, "READ_DATA": 4, "SERVICE": 4, "gps": 4, "SYSTEM_UI_VISIBILITY_EXTENSION": 4, "AUDIO_FILE_ACCESS": 4, "AM_COMMUNICATION": 4, "FILE": 4, "READ_PROVIDER": 4, "changed": 4, "PUSH_RECEIVED": 4, "PROVIDER_ACCESS_MODIFY_CONFIGURATION": 3, "STATUS_BAR_SERVICE": 3, "CALL_AMAZON_DEVICE_INFORMATION_PROVIDER": 3, "BIND": 3, "REMOTE_API": 3, "MDM_APP_MGMT": 3, "WRITE_IMAGES": 3, "USB_PERMISSION": 3, "internal": 3, "DIAL_PHONE": 3, "RECEIVE_MESSAGE": 3, "REGISTRATION": 3, "SUPPORT_PERMISSION": 3, "NOTIFICATION": 3, "location": 3, "RECIEVE_MCS_MESSAGE": 3, "SANDBOX": 3, "FORCE_STOP_PACKAGE": 3, "WRITE_DATA": 3, "BLABLACONNECT": 3, "CONTENT_PROVIDER": 3, "MDM_RESTRICTION": 3, "ACCESS_DATA": 3, "PAYMENT_BROADCAST": 3, "Controller": 3, "CONTROL_SERVICE": 3, "READ_MEDIA_STORAGE": 3, "siji": 2, "INNER_MESSAGE": 2, "PROFILER": 2, "tsapp": 2, "NEWSCOUNT": 2, "CROSS_PROCESS_BROADCAST_MANAGER": 2, "NFC_SE": 2, "ACCESS_START_SVC": 2, "gcm_broadcasting": 2, "PERMISSION": 2, "BEEM_SERVICE": 2, "MANAGE_OVERLAY_PERMISSION": 2, "write": 2, "WDID": 2, "SMALLAPP": 2, "ENTERPRISE_DEVICE_ADMIN": 2, "ACCOUNT_AUTHENTICATION_CHANGE": 2, "MDM_FIREWALL": 2, "AGAPAO_INTERNET": 2, "FILE_ACCESS": 2, "ACCESS_NETWORK_STAT": 2, "PUSHIO_MESSAGE": 2, "MDM_EXCHANGE": 2, "AVIATE_RECEIVE": 2, "QUICK_OPTIMIZE": 2, "ADDON": 2, "signature": 2, "client": 2, "PASSLOCK_END_BY_BACKPRESSED": 2, "SET_PERMISSION": 2, "BIND_JOB_SERVICE": 2, "BOOK_PROCESS_STATE": 2, "hit": 2, "message": 2, "BIND_DATA_CONSUMER": 2, "GENERAL": 2, "EXTENSION_PERMISSION": 2, "testapp": 2, "RECEIVE_DATA": 2, "MOUNT_UNMOUNT_FILESYSTEM": 2, "DOWNLOAD_CALLBACK_SVC": 2, "ACCESS_PERMISSION": 2, "READ_ALARM": 2, "READ_CONFIG": 2, "MEDIA_MOUNTED": 2, "ACCESS_NOTIFICATIONS": 2, "RECEIVE_BROADCAST": 2, "RECEIVER": 2, "AUTO_SEND": 2, "WRITE_SDCARD": 2, "SERVICE_aff39f097fd6dfa17028673b24868eee7211cb86": 2, "RECEIVE_SCHEDULE_FINISH": 2, "MultiDexApplication": 2, "integrate": 2, "FORE_SERVICE": 2, "soccerbeach": 2, "READ_EXTENSION_DATA": 2, "SYNC_STATUS": 2, "BACK_ENGINE": 2, "alihb": 2, "NEW_OUTGOING_CALL": 2, "READ_RESULTS": 2, "WIFI_STATE": 2, "BROWSER": 2, "INNER_BROCAST": 2, "SEND_BROADCAST": 2, "DIAL": 2, "ACTION_COMPLAINT_REPLY": 2, "zixun": 2, "CONTENT_PROVIDER_ACCESS": 2, "RESANA_ADS": 2, "INTERNAL_ACCESS_ONLY": 2, "MANAGE_OWN_CALLS": 2, "PUSH_SERVICE": 2, "push": 2, "ACCESS_SYN_SERVICE": 2, "READ_PRELOAD_DEVICE_INFO_PROVIDER": 2, "LAUNCH": 2, "iap": 2, "CAN_CALL_MAP_INFORMATION_PROVIDER": 2, "CryptoAllPermission": 2, "FLAG_ACTIVITY_NEW_TASK": 2, "CAR_MODE_SERVICE_STATUS_RECEIVED": 2, "STORE_ACCESS": 2, "LOGOUT": 2, "SHOW_LOCATION": 2, "AUTH_SDK": 2, "ReadSharedData2": 2, "RECEIVED_BOOT_COMPLETED": 2, "theme": 2, "READ_CLOCK": 2, "MDM_KIOSK_MODE": 2, "WRITE_CAMCARD_LITE": 2, "FM_RADIO_TRANSMITTER": 2, "FREQUENCY_CAP_SECURITY": 2, "kecheng": 2, "SESSION_EXPIRE": 2, "girlscalendar": 2, "updatelocation": 2, "SINA_PUSH": 2, "READ_MY_DATA": 2, "DATACENTER": 2, "FM_RADIO_RECEIVER": 2, "FRAMEWORK_SECURITY": 2, "CHAT_INFO_ACCESS": 2, "STORGE": 2, "CONFIG_PERMISSION": 2, "provider": 2, "ACCESS_KUGOU_SERVICE": 2, "innoplus": 2, "AUTHENTICATE": 2, "VIEW": 2, "WRITE_INTERNAL_STROAGE": 2, "INSTALL_REFERRER": 2, "socketevent": 2, "READ_CAMCARD_LITE": 2, "YIXIN_SDK_MESSAGE": 2, "SPEECH_RECOGNIZER": 2, "AOM_RECEIVE": 2, "LAUNCH_PEN_AND_PDF_FILE_CHOOSER": 2, "SYNC_DATA": 2, "COLLECTOR": 2}
benign_api_calls = {"Ljava/util/regex/MatchResult;->start": 718, "Landroid/bluetooth/le/BluetoothLeScanner;->startScan": 592, "Landroid/bluetooth/le/BluetoothLeScanner;->stopScan": 569, "Landroid/bluetooth/le/ScanCallback;-><init>": 536, "Landroid/bluetooth/BluetoothAdapter;->getBluetoothLeScanner": 534, "Landroid/bluetooth/le/ScanSettings$Builder;-><init>": 524, "Landroid/bluetooth/le/ScanSettings$Builder;->build": 524, "Landroid/bluetooth/le/ScanResult;->getScanRecord": 524, "Landroid/bluetooth/le/ScanSettings$Builder;->setScanMode": 519, "Ljava/lang/Math;->nextAfter": 508, "Landroid/bluetooth/le/ScanResult;->getRssi": 494, "Landroid/bluetooth/le/ScanRecord;->getBytes": 491, "[Lcom/facebook/ads/AdSettings$TestAdType;->clone": 453, "Landroid/media/session/MediaSession;->getCurrentControllerInfo": 444, "Ljava/util/LinkedHashSet;->removeAll": 438, "Ljava/util/concurrent/LinkedBlockingDeque;->clear": 435, "[Lcom/facebook/ads/VideoAutoplayBehavior;->clone": 427, "Landroid/media/session/MediaSessionManager$RemoteUserInfo;->getPid": 423, "Landroid/media/session/MediaSessionManager$RemoteUserInfo;->getUid": 423, "Landroid/media/session/MediaSessionManager$RemoteUserInfo;->getPackageName": 423, "Landroid/service/media/MediaBrowserService;->getCurrentBrowserInfo": 421, "[Lcom/facebook/ads/VideoStartReason;->clone": 418, "Landroid/media/session/MediaSessionManager;->isTrustedForMediaControl": 418, "Ljava/util/concurrent/LinkedBlockingDeque;->toArray": 416, "Landroid/bluetooth/le/ScanCallback;->onScanResult": 407, "Landroid/bluetooth/le/ScanCallback;->onScanFailed": 406, "[Lcom/facebook/appevents/internal/AppEventsLoggerUtility$GraphAPIActivityType;->clone": 400, "[Lcom/facebook/ads/CacheFlag;->clone": 399, "[Lcom/facebook/share/internal/CameraEffectFeature;->clone": 395, "Landroid/bluetooth/le/ScanCallback;->onBatchScanResults": 387, "Landroid/bluetooth/le/ScanSettings$Builder;->setReportDelay": 384, "Landroid/bluetooth/le/BluetoothLeScanner;->flushPendingScanResults": 374, "[Lcom/facebook/places/model/CurrentPlaceRequestParams$ScanMode;->clone": 360, "[Lcom/facebook/places/model/CurrentPlaceRequestParams$ConfidenceLevel;->clone": 360, "[Lcom/facebook/places/PlaceManager$LocationError;->clone": 360, "[Lcom/facebook/places/internal/ScannerException$Type;->clone": 360, "[Lcom/facebook/share/model/ShareMessengerURLActionButton$WebviewHeightRatio;->clone": 349, "[Lcom/facebook/share/model/ShareMessengerGenericTemplateContent$ImageAspectRatio;->clone": 349, "[Lcom/facebook/share/model/ShareMessengerMediaTemplateContent$MediaType;->clone": 349, "[Lcom/google/android/exoplayer2/Timeline;->clone": 337, "[Lkotlin/LazyThreadSafetyMode;->clone": 336, "Landroid/hardware/Camera$Parameters;->unflatten": 335, "[Lkotlin/reflect/KParameter$Kind;->clone": 334, "[Lkotlin/text/RegexOption;->clone": 333, "[Lkotlin/DeprecationLevel;->clone": 331, "[Lkotlin/io/FileWalkDirection;->clone": 330, "Ljava/util/concurrent/locks/ReentrantReadWriteLock;->getWriteHoldCount": 330, "[Lkotlin/annotation/AnnotationRetention;->clone": 329, "[Lkotlin/annotation/AnnotationTarget;->clone": 329, "[Lkotlin/text/CharCategory;->clone": 327, "[Lkotlin/text/CharDirectionality;->clone": 327, "[Lkotlin/io/OnErrorAction;->clone": 327, "[Lkotlin/reflect/KVisibility;->clone": 326, "[Landroid/arch/lifecycle/c$b;->clone": 325, "[Landroid/arch/lifecycle/c$a;->clone": 325, "Ljava/lang/String;->codePointBefore": 323, "[Lcom/facebook/ads/AudienceNetworkActivity$Type;->clone": 323, "[Lkotlin/collections/State;->clone": 321, "[Lkotlin/reflect/KVariance;->clone": 317, "Ljava/util/concurrent/TimeUnit;->hashCode": 316, "[Lio/reactivex/internal/disposables/EmptyDisposable;->clone": 314, "[Lio/reactivex/internal/disposables/DisposableHelper;->clone": 314, "[Lcom/facebook/ads/internal/e/f$a;->clone": 310, "Ljava/util/AbstractMap;->size": 306, "[Lio/reactivex/BackpressureStrategy;->clone": 304, "[Lio/reactivex/internal/util/NotificationLite;->clone": 304, "[Lio/reactivex/internal/subscriptions/SubscriptionHelper;->clone": 297, "[Lio/reactivex/internal/util/ErrorMode;->clone": 294, "[Lio/reactivex/internal/subscriptions/EmptySubscription;->clone": 290, "[Lio/reactivex/internal/util/ArrayListSupplier;->clone": 289, "Landroid/hardware/Camera$Parameters;->getMaxNumMeteringAreas": 289, "[Lio/reactivex/internal/operators/flowable/FlowableInternalHelper$RequestMax;->clone": 285, "[Lcom/facebook/ads/internal/protocol/AdErrorType;->clone": 285, "Landroid/hardware/Camera$Parameters;->setMeteringAreas": 284, "Ljava/lang/reflect/UndeclaredThrowableException;->getCause": 284, "[Lio/reactivex/internal/util/EmptyComponent;->clone": 282, "[Lio/reactivex/internal/util/HashMapSupplier;->clone": 282, "Landroid/widget/ArrayAdapter;->addAll": 277, "[Lio/reactivex/annotations/BackpressureKind;->clone": 275, "Landroid/media/PlaybackParams;-><init>": 275, "[Lio/reactivex/BackpressureOverflowStrategy;->clone": 274, "[Lio/reactivex/internal/operators/maybe/MaybeToPublisher;->clone": 272, "[Lkotlin/internal/RequireKotlinVersionKind;->clone": 272, "Lcom/google/firebase/database/Query;->endAt": 270, "Lcom/google/firebase/database/Query;->startAt": 270, "[Lio/reactivex/internal/functions/Functions$HashSetCallable;->clone": 270, "Lcom/google/firebase/database/Query;->equalTo": 270, "[Lio/reactivex/internal/functions/Functions$NaturalComparator;->clone": 268, "[Lio/reactivex/observers/TestObserver$EmptyObserver;->clone": 267, "Landroid/graphics/Path;->set": 265, "Ljava/util/ArrayDeque;->offerFirst": 263, "Landroid/app/DialogFragment;->onAttach": 261, "[Lio/reactivex/internal/operators/single/SingleInternalHelper$ToFlowable;->clone": 260, "[Lio/reactivex/internal/operators/single/SingleInternalHelper$NoSuchElementCallable;->clone": 259, "[Lio/reactivex/subscribers/TestSubscriber$EmptySubscriber;->clone": 259, "[Lio/reactivex/internal/util/ListAddBiConsumer;->clone": 258, "[Lio/reactivex/internal/operators/single/SingleInternalHelper$ToObservable;->clone": 258, "[Lkotlin/Experimental$Level;->clone": 258, "[Lcom/facebook/ads/internal/protocol/e;->clone": 258, "[Lio/reactivex/internal/operators/observable/ObservableInternalHelper$MapToInt;->clone": 258, "[Lcom/facebook/ads/internal/protocol/AdPlacementType;->clone": 255, "[Lio/reactivex/observers/BaseTestConsumer$TestWaitStrategy;->clone": 250, "Ljava/util/concurrent/atomic/AtomicLongArray;->decrementAndGet": 250, "[Lcom/facebook/appevents/codeless/internal/EventBinding$ActionType;->clone": 250, "[Lcom/facebook/ads/internal/settings/a$a;->clone": 248, "[Lcom/facebook/ads/NativeAdBase$MediaCacheFlag;->clone": 248, "[Lio/reactivex/parallel/ParallelFailureHandling;->clone": 248, "Lcom/fasterxml/jackson/core/JsonGenerator;->writeNumber": 248, "[Lcom/facebook/appevents/codeless/internal/EventBinding$MappingMethod;->clone": 248, "[Lcom/facebook/appevents/codeless/internal/PathComponent$MatchBitmaskType;->clone": 248, "Landroid/security/KeyPairGeneratorSpec$Builder;->setSubject": 246, "[Lcom/facebook/ads/internal/protocol/d;->clone": 246, "Landroid/security/KeyPairGeneratorSpec$Builder;->setAlias": 246, "Landroid/security/KeyPairGeneratorSpec$Builder;->build": 246, "Landroid/security/KeyPairGeneratorSpec$Builder;-><init>": 246, "Landroid/security/KeyPairGeneratorSpec$Builder;->setSerialNumber": 245, "[Lcom/facebook/ads/NativeBannerAdView$Type;->clone": 245, "[Lcom/facebook/ads/NativeAdBase$NativeComponentTag;->clone": 244, "Landroid/security/KeyPairGeneratorSpec$Builder;->setEndDate": 244, "Landroid/security/KeyPairGeneratorSpec$Builder;->setStartDate": 244, "Landroid/media/AudioTrack;->setPlaybackParams": 244, "Landroid/graphics/Path;->op": 242, "[Lcom/facebook/ads/internal/m/e;->clone": 237, "[Landroidx/lifecycle/Lifecycle$State;->clone": 235, "[Landroidx/lifecycle/Lifecycle$Event;->clone": 232, "Landroid/database/sqlite/SQLiteDatabase;->yieldIfContendedSafely": 230, "Landroid/bluetooth/le/ScanResult;->getTimestampNanos": 229, "Landroid/view/TextureView;->setLayoutParams": 225, "Landroid/hardware/Camera$Parameters;->getVideoStabilization": 224, "[Landroidx/annotation/RestrictTo$Scope;->clone": 222, "[Landroidx/loader/content/ModernAsyncTask$Status;->clone": 221, "[Lcom/viewpagerindicator/TitlePageIndicator$IndicatorStyle;->clone": 221, "Landroid/util/JsonToken;->ordinal": 218, "Landroid/util/JsonToken;->values": 218, "Landroid/transition/TransitionSet;->setDuration": 216, "[Lcom/viewpagerindicator/TitlePageIndicator$LinePosition;->clone": 213, "Landroid/widget/TextView;->setKeyListener": 211, "Landroid/hardware/camera2/CameraDevice;->createCaptureSession": 210, "Landroid/hardware/camera2/CameraManager;->openCamera": 209, "Landroid/net/NetworkCapabilities;->hasTransport": 209, "Landroid/hardware/camera2/CaptureRequest$Builder;->addTarget": 208, "Landroid/hardware/camera2/CameraDevice$StateCallback;-><init>": 208, "Landroid/hardware/camera2/CameraDevice;->createCaptureRequest": 208, "Landroid/hardware/camera2/CameraCaptureSession$StateCallback;-><init>": 208, "Landroid/hardware/camera2/CaptureRequest$Builder;->build": 208, "Landroid/hardware/camera2/CameraDevice;->close": 207, "Landroid/hardware/camera2/CaptureRequest$Builder;->set": 206, "[Lcom/facebook/ads/internal/a/a;->clone": 204, "[Lcom/google/android/gms/internal/measurement/zzxs;->clone": 204, "Landroid/transition/TransitionSet;->setInterpolator": 204, "[Lcom/google/android/gms/internal/measurement/zzux;->clone": 204, "Ljava/util/concurrent/atomic/AtomicLongFieldUpdater;->getAndSet": 204, "Lcom/google/android/gms/internal/zzbgo;->zza": 204, "[Lcom/google/android/gms/internal/measurement/zzui;->clone": 204, "[Lcom/google/android/gms/internal/measurement/zzxx;->clone": 204, "[Lcom/facebook/ads/internal/a/d;->clone": 204, "Landroid/hardware/camera2/CameraCaptureSession;->setRepeatingRequest": 202, "Landroid/graphics/drawable/StateListDrawable;->setVisible": 201, "Landroid/graphics/drawable/StateListDrawable;->isVisible": 201, "[Lcom/google/android/gms/internal/measurement/zzuk;->clone": 200, "Landroid/animation/AnimatorListenerAdapter;->onAnimationCancel": 200, "[Lcom/facebook/share/internal/ShareStoryFeature;->clone": 198, "Ljava/lang/Throwable;->hashCode": 198, "Landroid/bluetooth/le/ScanResult;->getDevice": 197, "[Lcom/mopub/mobileads/AdTypeTranslator$CustomEventType;->clone": 194, "[Lcom/mopub/common/util/Drawables;->clone": 193, "[Lcom/mopub/common/LocationService$LocationAwareness;->clone": 192, "[Lcom/mopub/common/util/DeviceUtils$IP;->clone": 192, "[Lcom/mopub/common/util/ResponseHeader;->clone": 190, "[Lcom/mopub/volley/Request$Priority;->clone": 190, "[Lcom/mopub/common/ClientMetadata$MoPubNetworkType;->clone": 189, "Ljava/util/concurrent/atomic/AtomicLongFieldUpdater;->weakCompareAndSet": 187, "[Lcom/mopub/common/MoPub$LocationAwareness;->clone": 187, "[Lcom/mopub/common/AdFormat;->clone": 186, "[Lcom/mopub/common/CloseableLayout$ClosePosition;->clone": 185, "[Lcom/mopub/mraid/ViewState;->clone": 185, "[Lcom/mopub/mraid/MraidJavascriptCommand;->clone": 185, "[Lcom/mopub/mraid/PlacementType;->clone": 185, "Ljava/util/logging/Logger;->setUseParentHandlers": 185, "Landroid/hardware/Camera$Parameters;->getFocusAreas": 185, "[Lcom/mopub/network/MoPubNetworkError$Reason;->clone": 184, "[Lcom/mopub/common/util/DeviceUtils$ForceOrientation;->clone": 181, "[Lcom/mopub/common/UrlAction;->clone": 181, "[Lcom/mopub/common/CreativeOrientation;->clone": 181, "[Lcom/facebook/ads/internal/p/a/j;->clone": 181, "[Lcom/mopub/mobileads/VastErrorCode;->clone": 180, "Landroid/hardware/Camera$Parameters;->getPreviewFpsRange": 179, "[Lcom/google/common/collect/MapConstraints$NotNullMapConstraint;->clone": 179, "[Lcom/mopub/nativeads/NativeErrorCode;->clone": 179, "Landroid/hardware/Camera$Parameters;->getColorEffect": 178, "Landroid/widget/TimePicker;->getHour": 178, "Landroid/widget/TimePicker;->setHour": 176, "[Lcom/facebook/ads/internal/q/b/b;->clone": 175, "[Lcom/mopub/nativeads/RequestParameters$NativeAdAsset;->clone": 175, "[Lcom/facebook/ads/internal/q/a/c;->clone": 175, "[Lcom/facebook/ads/internal/j/a$a;->clone": 174, "[Lcom/facebook/ads/internal/m/f;->clone": 174, "Landroid/hardware/Camera$Parameters;->getMeteringAreas": 173, "[Lcom/facebook/ads/internal/h/e;->clone": 173, "[Lcom/facebook/ads/internal/c/a$c;->clone": 173}
print(len(benign_permissions))
print(len(benign_api_calls))

200
200


In [ ]:
benign_API_calls_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Androguard+Predefined_Dictionary/APKs_Properties_Files/Properties_count/Benign_API_Calls_Dictionary'
benign_permissions_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Androguard+Predefined_Dictionary/APKs_Properties_Files/Properties_count/Benign_permission_Dictionary'

normal_permissions = {}
for perm in open(benign_permissions_path, 'r'):
    normal_permissions = perm
    
normal_api_calls = {}
for api_calls in open(benign_API_calls_path, 'r'):
    normal_api_calls = api_call

In [ ]:
print(len(shas_100_list))
print(shas_100_list[0])

5518
AB4C1893A715230464EAA41C9540975F66978A900305662220656F2E885EEA49


In [ ]:
start_time = time.time()

test_apks = [] # binary matrix of whole malware dataset
test_directory_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/Inter_Linear/11k_100_AEs_Testing/11k_100_APKs_AEs_Testing_Properties_Files/'
test_shas = os.listdir('/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/Inter_Linear/11k_100_AEs_Testing/11k_100_APKs_AEs_Testing_Properties_Files/Test_APKs_App_Components/')

#test_directory_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/APKs_Properties_Files/'

count=0
for sha in test_shas:
    apk_vector = np.zeros(65536, dtype=np.int16) # vectors representing an APK
    apk_features = reading_features(test_directory_path, sha) # add benign directry app's features to list
    
    apk_indices = []
    for feature in apk_features:
        try:
            apk_indices.append(unique_features.index(feature))
        except:
            continue
    '''
    #Adding perturbation from collected bening permissions.
    for perm in list(benign_permissions.keys()):
        if(perm in unique_features):
            index = unique_features.index(perm)
            apk_indices.append(index)
    
    #Adding perturbation from collected bening API Calls.
    for api_call in list(benign_api_calls.keys()):
        if(api_call in unique_features):
            index = unique_features.index(api_call)
            apk_indices.append(index)
    '''
    apk_vector[apk_indices] = 1
    apk_vector = apk_vector.reshape(256,256)
    #apk_indices = [unique_features.index(feature) for feature in apk_features] # 
    
    test_apks.append(apk_vector)
    if(count%50 == 0):
        print(count)
    count+=1

test_apks = np.array(test_apks)
print(test_apks.shape)

print("Time taken by extracting features from benign apps is ", (time.time()-start_time)/60.0, ' minutes.' )

0
50
(100, 256, 256)
Time taken by extracting features from benign apps is  0.770036534468333  minutes.


In [ ]:
np.save('/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/Inter_Linear/11k_100_AEs_Testing/AEs_for_testing_200/100_same_APKs_properties_Files/Normal_100_diff.npy', test_apks)

**Adding Perturbtion via Attack2**

In [ ]:
def adding_perturbation(prtrb_count):
  generated_images_path = '/content/drive/My Drive/Research-Adverserial_ML/AEs_Generations/AEs_with_GANs/Samples_Generated_DCGAN/DCGAN_11K/'
  generated_benign = cv2.imread(os.path.join(generated_images_path + '11k1000__0.jpg'))
  
  b_gen, g_gen, r_gen    = generated_benign[:, :, 0], generated_benign[:, :, 1], generated_benign[:, :, 2]
    
  #perturb_pixels_red = random.sample(list(r_gen.flatten()), len(r_gen.flatten()))
  perturb_pixels_red = list(r_gen.flatten())

  for pixel in perturb_pixels_red:
    prprty = lookUpTable_red[pixel]
    if(prprty in unique_features):
      index = unique_features.index(prprty)
      apk_indices.append(index)

  perturb_property = []
  i=0
  while(len(perturb_property) <prtrb_count and i<256):
    prprty = lookUpTable_red[perturb_pixels_red[i]]
    perturb_property.append(prprty)
    i+=1
  return perturb_property

In [ ]:
unique_features = []
for feature in open('/content/drive/My Drive/Research-Adverserial_ML/P9:Malware_Detection_Based_on_CNN/Datasets/Unique_features', 'r'):
  unique_features.append(feature.rstrip())
print(len(unique_features))

65536


In [ ]:
start_time = time.time()
import cv2

test_apks = [] # binary matrix of whole malware dataset
test_directory_path = '/content/drive/My Drive/Research-Adverserial_ML/AEs_Generations/AEs_with_GANs/Samples_Generated_DCGAN/AEs/Testing_forEvasion_Samples/100_PropertiesFiles_for_P9_fromSameTraing_DataSet/'
test_shas = os.listdir('/content/drive/My Drive/Research-Adverserial_ML/AEs_Generations/AEs_with_GANs/Samples_Generated_DCGAN/AEs/Testing_forEvasion_Samples/100_PropertiesFiles_for_P9_fromSameTraing_DataSet/Test_APKs_App_Components/')

generated_images_path = '/content/drive/My Drive/Research-Adverserial_ML/AEs_Generations/AEs_with_GANs/Samples_Generated_DCGAN/DCGAN_11K/'
generated_benign = cv2.imread(os.path.join(generated_images_path + '11k1000__0.jpg'))

b_gen, g_gen, r_gen    = generated_benign[:, :, 0], generated_benign[:, :, 1], generated_benign[:, :, 2]
  
#perturb_pixels_red = random.sample(list(r_gen.flatten()), len(r_gen.flatten()))
perturb_pixels_red = list(r_gen.flatten())
perturb_pixels_green = list(g_gen.flatten())
perturb_pixels_blue = list(b_gen.flatten())


#test_directory_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/APKs_Properties_Files/'

count=0
prtrbtn_count = 0
prtrbtn_prcntg = int(5.0*655.36/3.0)

for sha in test_shas:
    apk_vector = np.zeros(65536, dtype=np.int16) # vectors representing an APK
    apk_features = reading_features(test_directory_path, sha) # add benign directry app's features to list
    
    apk_indices = []
    for feature in apk_features:
        #try:
        apk_indices.append(unique_features.index(feature))
        #except:
        #    continue
    '''
    #Adding pertubation from red channel
    channl_prtrb_count = 0
    for pixel in perturb_pixels_red:
      prprty = lookUpTable_red[pixel]
      if(prprty in unique_features and channl_prtrb_count <= prtrbtn_prcntg):
        index = unique_features.index(prprty)
        apk_indices.append(index)
        channl_prtrb_count += 1

    #Adding perturbation from green channel
    channl_prtrb_count = 0
    for pixel in perturb_pixels_green:
      prprty = lookUpTable_green[pixel]
      if(prprty in unique_features and channl_prtrb_count <= prtrbtn_prcntg):
        index = unique_features.index(prprty)
        apk_indices.append(index)
        channl_prtrb_count += 1

    #Adding perturbation from blue channel
    channl_prtrb_count = 0
    for pixel in perturb_pixels_blue:
      prprty = lookUpTable_blue[pixel]
      if(prprty in unique_features and channl_prtrb_count <= prtrbtn_prcntg):
        index = unique_features.index(prprty)
        apk_indices.append(index)
        channl_prtrb_count += 1
    '''
    apk_vector[apk_indices] = 1
    apk_vector = apk_vector.reshape(256,256)
    test_apks.append(apk_vector)

test_apks = np.array(test_apks)
print(test_apks.shape)

print("Time taken by extracting features from benign apps is ", (time.time()-start_time)/60.0, ' minutes.' )

ValueError: ignored

In [ ]:
np.save('/content/drive/My Drive/Research-Adverserial_ML/AEs_Generations/AEs_with_GANs/Samples_Generated_DCGAN/AEs/Attack_On_P9:Malware_Detection_BasedOn_CNN/100NormalSameTrainigDataset.npy', test_apks)

**LookUp Tables**

In [ ]:
lookUpTable_red = {0: 'Ljava/lang/Long;->valueOf',
 1: 'Ljava/util/Collections;->sort',
 2: 'Landroid/view/Gravity;->apply',
 3: 'Ljava/io/File;->listFiles',
 4: 'Landroid/view/animation/DecelerateInterpolator;-><init>',
 5: 'Landroid/content/Intent;->putExtra',
 6: 'Landroid/view/MotionEvent;->getX',
 7: 'Landroid/view/MotionEvent;->getY',
 8: 'Landroid/app/PendingIntent;->getActivities',
 9: 'Landroid/os/Handler;-><init>',
 10: 'Ljava/lang/StringBuilder;-><init>',
 11: 'Landroid/webkit/WebView;->getSettings',
 12: 'Landroid/view/MotionEvent;->getAction',
 13: 'Landroid/widget/Toast;->makeText',
 14: 'Ljava/security/MessageDigest;->getInstance',
 15: 'Ljava/util/Locale;->getDefault',
 16: 'Ljava/util/ArrayList;->size',
 17: 'Ljava/io/ByteArrayInputStream;-><init>',
 18: 'Ljava/util/WeakHashMap;-><init>',
 19: 'Ljava/lang/Class;->newInstance',
 20: 'Ljava/lang/Object;->hashCode',
 21: 'Ljava/io/File;->getName',
 22: 'Landroid/graphics/Canvas;-><init>',
 23: 'Landroid/app/Activity;->onResume',
 24: 'Landroid/widget/EditText;-><init>',
 25: 'Ljava/security/MessageDigest;->digest',
 26: 'Ljava/util/concurrent/Executors;->newFixedThreadPool',
 27: 'Lorg/json/JSONObject;-><init>',
 28: 'Ljava/security/MessageDigest;->update',
 29: 'Landroid/os/Handler;->obtainMessage',
 30: 'Landroid/graphics/Canvas;->clipRect',
 31: 'Ljava/util/concurrent/ConcurrentHashMap;-><init>',
 32: 'Landroid/net/Uri;->encode',
 33: 'Landroid/content/res/Resources;->getString',
 34: 'Ljava/io/BufferedOutputStream;-><init>',
 35: 'Ljava/util/concurrent/atomic/AtomicBoolean;-><init>',
 36: 'Ljava/lang/Thread;-><init>',
 37: 'Ljava/lang/reflect/Modifier;->isStatic',
 38: 'Landroid/app/Notification$BigTextStyle;->bigText',
 39: 'Landroid/graphics/Color;->argb',
 40: 'Landroid/os/Bundle;->putSparseParcelableArray',
 41: 'Landroid/graphics/Bitmap;->getWidth',
 42: 'Ljava/lang/Runnable;->run',
 43: 'Ljava/lang/Object;->getClass',
 44: 'Landroid/widget/Button;->setText',
 45: 'Landroid/widget/TextView;-><init>',
 46: 'Landroid/os/Bundle;->getString',
 47: 'Ljava/lang/Character;->getDirectionality',
 48: 'Landroid/view/ViewGroup$MarginLayoutParams;-><init>',
 49: 'Ljava/util/concurrent/atomic/AtomicInteger;-><init>',
 50: 'Landroid/app/Activity;->getParentActivityIntent',
 51: 'Landroid/widget/Scroller;-><init>',
 52: 'Landroid/content/Context;->obtainStyledAttributes',
 53: 'Landroid/widget/TextView;->setSingleLine',
 54: 'Ljava/lang/IllegalStateException;-><init>',
 55: 'Ljava/util/ArrayList;->toArray',
 56: 'Ljava/util/Locale;-><init>',
 57: 'Landroid/provider/Settings$Secure;->getString',
 58: 'Landroid/view/accessibility/AccessibilityNodeInfo;->setLabelFor',
 59: 'Ljava/util/LinkedList;->remove',
 60: 'Landroid/content/IntentFilter;-><init>',
 61: 'Landroid/widget/RelativeLayout;->addView',
 62: 'Ljava/lang/Integer;->valueOf',
 63: 'Landroid/content/pm/PackageManager;->getResourcesForApplication',
 64: 'Ljava/text/SimpleDateFormat;-><init>',
 65: 'Ljava/lang/String;->split',
 66: 'Landroid/util/Base64;->decode',
 67: 'Ljava/lang/String;-><init>',
 68: 'Landroid/app/Activity;->startActivity',
 69: 'Ljava/util/Set;->contains',
 70: 'Lorg/json/JSONObject;->put',
 71: 'Lorg/json/JSONObject;->toString',
 72: 'Landroid/graphics/Rect;-><init>',
 73: 'Ljava/io/IOException;-><init>',
 74: 'Landroid/text/SpannableStringBuilder;->append',
 75: 'Ljava/util/Arrays;->hashCode',
 76: 'Ljavax/net/ssl/SSLSocketFactory;->createSocket',
 77: 'Ljava/util/Map;->keySet',
 78: 'Landroid/net/ConnectivityManager;->getActiveNetworkInfo',
 79: 'Landroid/os/Handler;->hasMessages',
 80: 'Ljava/lang/Math;->round',
 81: 'Ljava/util/BitSet;->set',
 82: 'Landroid/view/View$BaseSavedState;-><init>',
 83: 'Ljava/net/HttpURLConnection;->getHeaderField',
 84: 'Ljava/lang/StringBuilder;->append',
 85: 'Ljava/lang/Class;->getName',
 86: 'Landroid/widget/FrameLayout;-><init>',
 87: 'Landroid/widget/ProgressBar;-><init>',
 88: 'Ljava/lang/Throwable;->printStackTrace',
 89: 'Landroid/os/Handler;->postDelayed',
 90: 'Landroid/content/SharedPreferences$Editor;->putString',
 91: 'Landroid/media/RemoteControlClient;->setPlaybackState',
 92: 'Landroid/widget/FrameLayout$LayoutParams;-><init>',
 93: 'Landroid/view/WindowManager$LayoutParams;-><init>',
 94: 'Ljava/lang/Math;->abs',
 95: 'Ljava/lang/RuntimeException;-><init>',
 96: 'Ljava/io/File;->getAbsolutePath',
 97: 'Landroid/os/HandlerThread;-><init>',
 98: 'Landroid/view/ViewGroup;->addView',
 99: 'Landroid/content/Intent;->getStringExtra',
 100: 'Landroid/view/accessibility/AccessibilityNodeInfo;->obtain',
 101: 'Ljava/io/PrintWriter;->println',
 102: 'Ljava/util/Date;-><init>',
 103: 'Ljava/io/FileOutputStream;->close',
 104: 'Ljava/util/concurrent/FutureTask;-><init>',
 105: 'Landroid/widget/Button;-><init>',
 106: 'Landroid/os/Parcel;->readBundle',
 107: 'Landroid/widget/ImageView;-><init>',
 108: 'Ljava/lang/Math;->min',
 109: 'Landroid/graphics/BitmapFactory;->decodeResource',
 110: 'Ljava/lang/Math;->max',
 111: 'Lorg/json/JSONObject;->length',
 112: 'Ljava/nio/ByteBuffer;->get',
 113: 'Ljava/lang/StringBuilder;->insert',
 114: 'Ljava/lang/Math;->sin',
 115: 'Landroid/widget/TextView;->setVisibility',
 116: 'Ljava/lang/Throwable;-><init>',
 117: 'Ljava/lang/String;->toCharArray',
 118: 'Ljava/lang/String;->lastIndexOf',
 119: 'Landroid/os/Bundle;->getBoolean',
 120: 'Ljava/lang/IllegalArgumentException;-><init>',
 121: 'Landroid/widget/PopupWindow;-><init>',
 122: 'Ljava/util/concurrent/FutureTask;->get',
 123: 'Ljava/lang/Exception;-><init>',
 124: 'Ljava/io/FileOutputStream;->write',
 125: 'Ljava/lang/String;->toLowerCase',
 126: 'Ljava/util/ArrayList;->add',
 127: 'Ljava/io/FileOutputStream;-><init>',
 128: 'Landroid/util/Log;->e',
 129: 'Ljava/lang/ClassLoader;->loadClass',
 130: 'Ljava/io/File;-><init>',
 131: 'Ljava/util/ArrayList;-><init>',
 132: 'Landroid/util/Log;->i',
 133: 'Ljava/lang/Character;->codePointAt',
 134: 'Ljava/util/Arrays;->toString',
 135: 'Landroid/os/Parcel;->readString',
 136: 'Ljava/lang/Long;->toString',
 137: 'Ljava/nio/ByteBuffer;->put',
 138: 'Ljava/util/HashMap;-><init>',
 139: 'Ljava/io/PrintWriter;->print',
 140: 'Landroid/view/LayoutInflater;->inflate',
 141: 'Landroid/app/Dialog;->show',
 142: 'Landroid/graphics/RectF;-><init>',
 143: 'Ljava/io/ByteArrayOutputStream;->write',
 144: 'Landroid/app/Activity;->onDestroy',
 145: 'Landroid/graphics/BitmapFactory;->decodeStream',
 146: 'Landroid/util/Log;->w',
 147: 'Lorg/apache/http/client/HttpClient;->execute',
 148: 'Landroid/util/SparseArray;-><init>',
 149: 'Ljava/math/BigDecimal;-><init>',
 150: 'Landroid/content/Context;->startActivity',
 151: 'Landroid/graphics/Canvas;->save',
 152: 'Ljava/util/HashSet;-><init>',
 153: 'Landroid/content/ContentResolver;->query',
 154: 'Landroid/view/View;->requestFocus',
 155: 'Ljava/util/concurrent/ThreadPoolExecutor;-><init>',
 156: 'Ljava/util/Arrays;->copyOf',
 157: 'Ljava/lang/StringBuffer;-><init>',
 158: 'Ljava/lang/String;->format',
 159: 'Ljava/lang/reflect/Method;->invoke',
 160: 'Ljava/util/GregorianCalendar;-><init>',
 161: 'Landroid/os/Handler;->removeMessages',
 162: 'Landroid/widget/Scroller;->startScroll',
 163: 'Landroid/view/animation/PathInterpolator;-><init>',
 164: 'Ljava/util/TreeSet;-><init>',
 165: 'Landroid/os/Bundle;-><init>',
 166: 'Landroid/util/SparseArray;->get',
 167: 'Landroid/widget/RelativeLayout;-><init>',
 168: 'Landroid/webkit/WebView;-><init>',
 169: 'Landroid/content/res/Resources;->getColor',
 170: 'Ljava/lang/Object;->wait',
 171: 'Landroid/database/DataSetObservable;->unregisterObserver',
 172: 'Landroid/graphics/RectF;->set',
 173: 'Landroid/widget/ArrayAdapter;-><init>',
 174: 'Landroid/database/sqlite/SQLiteDatabase;->query',
 175: 'Landroid/view/accessibility/AccessibilityNodeInfo$CollectionItemInfo;->obtain',
 176: 'Ljava/io/File;->isFile',
 177: 'Landroid/content/Context;->getSystemService',
 178: 'Ljava/lang/Integer;->parseInt',
 179: 'Landroid/view/View;->getLayoutParams',
 180: 'Ljava/math/BigInteger;-><init>',
 181: 'Landroid/content/Context;->registerReceiver',
 182: 'Landroid/graphics/Paint;-><init>',
 183: 'Ljava/util/Arrays;->equals',
 184: 'Ljava/lang/Boolean;->toString',
 185: 'Landroid/view/View$AccessibilityDelegate;->performAccessibilityAction',
 186: 'Landroid/graphics/Bitmap;->createBitmap',
 187: 'Landroid/os/Bundle;->putParcelable',
 188: 'Landroid/view/animation/ScaleAnimation;->setInterpolator',
 189: 'Landroid/database/Cursor;->moveToPosition',
 190: 'Ljava/io/ByteArrayOutputStream;->toString',
 191: 'Landroid/view/View;-><init>',
 192: 'Landroid/app/AlertDialog$Builder;->setMessage',
 193: 'Landroid/view/LayoutInflater;->getFactory',
 194: 'Landroid/content/Context;->getPackageManager',
 195: 'Lorg/json/JSONArray;-><init>',
 196: 'Landroid/graphics/Point;-><init>',
 197: 'Landroid/os/ParcelFileDescriptor;->open',
 198: 'Landroid/widget/LinearLayout;-><init>',
 199: 'Ljava/io/File;->delete',
 200: 'Landroid/hardware/display/DisplayManager;->getDisplays',
 201: 'Landroid/view/ActionProvider;->onCreateActionView',
 202: 'Landroid/app/AlertDialog$Builder;->setPositiveButton',
 203: 'Ljava/util/regex/Pattern;->compile',
 204: 'Landroid/widget/LinearLayout$LayoutParams;-><init>',
 205: 'Landroid/hardware/fingerprint/FingerprintManager$CryptoObject;-><init>',
 206: 'Landroid/widget/PopupWindow;->update',
 207: 'Ljava/util/concurrent/CountDownLatch;->await',
 208: 'Landroid/graphics/drawable/Drawable;->setColorFilter',
 209: 'Ljava/io/InputStreamReader;-><init>',
 210: 'Landroid/view/ViewGroup$LayoutParams;-><init>',
 211: 'Ljava/util/Arrays;->fill',
 212: 'Ljava/nio/CharBuffer;->wrap',
 213: 'Ljava/lang/UnsupportedOperationException;-><init>',
 214: 'Landroid/content/Intent;-><init>',
 215: 'Landroid/view/View$MeasureSpec;->getMode',
 216: 'Landroid/view/Gravity;->applyDisplay',
 217: 'Ljava/lang/Long;->parseLong',
 218: 'Landroid/content/ContentValues;->put',
 219: 'Ljava/util/LinkedList;-><init>',
 220: 'Landroid/os/AsyncTask;-><init>',
 221: 'Landroid/app/AlertDialog$Builder;-><init>',
 222: 'Landroid/text/Html;->fromHtml',
 223: 'Ljava/util/concurrent/LinkedBlockingQueue;-><init>',
 224: 'Landroid/widget/TextView;->setTextAppearance',
 225: 'Ljava/util/LinkedHashMap;-><init>',
 226: 'Ljava/lang/String;->indexOf',
 227: 'Ljava/util/ArrayList;->remove',
 228: 'Ljava/util/List;->remove',
 229: 'Landroid/os/Handler;->postAtTime',
 230: 'Ljava/io/FileInputStream;->close',
 231: 'Ljava/lang/String;->valueOf',
 232: 'Landroid/content/Intent;->hasExtra',
 233: 'Ljava/util/Timer;-><init>',
 234: 'Landroid/app/Activity;->startActivityForResult',
 235: 'Landroid/view/MotionEvent;->obtain',
 236: 'Landroid/widget/FrameLayout;->addView',
 237: 'Landroid/text/util/Linkify;->addLinks',
 238: 'Lorg/json/JSONArray;->put',
 239: 'Landroid/widget/TextView;->setTextColor',
 240: 'Landroid/widget/TextView;->setText',
 241: 'Ljava/lang/String;->replace',
 242: 'Ljava/util/concurrent/atomic/AtomicReference;-><init>',
 243: 'Landroid/graphics/Canvas;->drawBitmap',
 244: 'Ljava/util/Arrays;->sort',
 245: 'Ljava/util/regex/Matcher;->group',
 246: 'Ljava/lang/String;->substring',
 247: 'Ljava/lang/NullPointerException;-><init>',
 248: 'Landroid/view/VelocityTracker;->getXVelocity',
 249: 'Landroid/view/View;->setTag',
 250: 'Landroid/graphics/drawable/Drawable$ConstantState;->newDrawable',
 251: 'Landroid/widget/EdgeEffect;->onPull',
 252: 'Ljava/io/OutputStream;->write',
 253: 'Landroid/app/AlertDialog$Builder;->setTitle',
 254: 'Ljava/io/FileInputStream;-><init>',
 255: 'Landroid/view/animation/AccelerateInterpolator;-><init>'}

In [ ]:
lookUpTable_blue = {0: 'CONTENT_PROVIDER_ACCESS',
 1: 'Ljava/util/Collections;->sort',
 2: 'Landroid/view/Gravity;->apply',
 3: 'Ljava/io/File;->listFiles',
 4: 'Landroid/view/animation/DecelerateInterpolator;-><init>',
 5: 'ACCESS_GPS',
 6: 'READ_STATE',
 7: 'FB_APP_COMMUNICATION',
 8: 'Landroid/app/PendingIntent;->getActivities',
 9: 'Landroid/os/Handler;-><init>',
 10: 'BLUETOOTH_PRIVILEGED',
 11: 'Landroid/webkit/WebView;->getSettings',
 12: 'Landroid/view/MotionEvent;->getAction',
 13: 'Landroid/widget/Toast;->makeText',
 14: 'PROCESS_PUSH_MSG',
 15: 'Ljava/util/Locale;->getDefault',
 16: 'Ljava/util/ArrayList;->size',
 17: 'INTERACT_ACROSS_USERS_FULL',
 18: 'Ljava/util/WeakHashMap;-><init>',
 19: 'Ljava/lang/Class;->newInstance',
 20: 'Ljava/lang/Object;->hashCode',
 21: 'ACCESS_THIRD_PARTY_APP_AUTHORIZATION',
 22: 'ACCESS_KUGOU_SERVICE',
 23: 'Landroid/app/Activity;->onResume',
 24: 'READ_APP_BADGE',
 25: 'Ljava/security/MessageDigest;->digest',
 26: 'Ljava/util/concurrent/Executors;->newFixedThreadPool',
 27: 'SERVICE',
 28: 'READ_SECURE_SETTINGS',
 29: 'Landroid/os/Handler;->obtainMessage',
 30: 'CHANGE_WIFI_STATE',
 31: 'Ljava/util/concurrent/ConcurrentHashMap;-><init>',
 32: 'RECEIVE_MyAction_1',
 33: 'Landroid/content/res/Resources;->getString',
 34: 'Ljava/io/BufferedOutputStream;-><init>',
 35: 'secoo',
 36: 'Ljava/lang/Thread;-><init>',
 37: 'Ljava/lang/reflect/Modifier;->isStatic',
 38: 'Landroid/app/Notification$BigTextStyle;->bigText',
 39: 'C2D_MESSAGE',
 40: 'Landroid/os/Bundle;->putSparseParcelableArray',
 41: 'REQUEST_IGNORE_BATTERY_OPTIMIZATIONS',
 42: 'Ljava/lang/Runnable;->run',
 43: 'Ljava/lang/Object;->getClass',
 44: 'ACCESS_SERVICE',
 45: 'WENDA_WRITE_PROVIDER',
 46: 'WRITE_CALENDAR',
 47: 'BADGE_COUNT_WRITE',
 48: 'READ_GSERVICES',
 49: 'Ljava/util/concurrent/atomic/AtomicInteger;-><init>',
 50: 'Landroid/app/Activity;->getParentActivityIntent',
 51: 'CHANGE_WIFI_MULTICAST_STATE',
 52: 'Landroid/content/Context;->obtainStyledAttributes',
 53: 'ACCESS_DATA',
 54: 'Ljava/lang/IllegalStateException;-><init>',
 55: 'store',
 56: 'Ljava/util/Locale;-><init>',
 57: 'Landroid/provider/Settings$Secure;->getString',
 58: 'BOOT_COMPLETED',
 59: 'Ljava/util/LinkedList;->remove',
 60: 'AUTH',
 61: 'FULL',
 62: 'Ljava/lang/Integer;->valueOf',
 63: 'CAPTURE_AUDIO_OUTPUT',
 64: 'Ljava/text/SimpleDateFormat;-><init>',
 65: 'Ljava/lang/String;->split',
 66: 'Landroid/util/Base64;->decode',
 67: 'Ljava/lang/String;-><init>',
 68: 'Landroid/app/Activity;->startActivity',
 69: 'Ljava/util/Set;->contains',
 70: 'Lorg/json/JSONObject;->put',
 71: 'Lorg/json/JSONObject;->toString',
 72: 'AUTHENTICATE_ACCOUNTS',
 73: 'REQUEST_IGNORE_BATTERY_OPTIMIZATIONS',
 74: 'Landroid/text/SpannableStringBuilder;->append',
 75: 'Ljava/util/Arrays;->hashCode',
 76: 'Ljavax/net/ssl/SSLSocketFactory;->createSocket',
 77: 'Ljava/util/Map;->keySet',
 78: 'SIGNAL_PERSISTENT_PROCESSES',
 79: 'Landroid/os/Handler;->hasMessages',
 80: 'Ljava/lang/Math;->round',
 81: 'WRITE_CALL_LOG',
 82: 'Landroid/view/View$BaseSavedState;-><init>',
 83: 'Ljava/net/HttpURLConnection;->getHeaderField',
 84: 'gps',
 85: 'HANDLE_PUSH',
 86: 'READ_INTERNAL_STORAGE',
 87: 'Landroid/widget/ProgressBar;-><init>',
 88: 'GOOGLE_AUTH',
 89: 'Landroid/os/Handler;->postDelayed',
 90: 'Landroid/content/SharedPreferences$Editor;->putString',
 91: 'PROVIDER_INSERT_BADGE',
 92: 'Landroid/widget/FrameLayout$LayoutParams;-><init>',
 93: 'SET_ALARM',
 94: 'BROADCAST_PACKAGE_INSTALL',
 95: 'RECEIVE_SMS',
 96: 'Ljava/io/File;->getAbsolutePath',
 97: 'ACCESS_WEATHERCLOCK_PROVIDER',
 98: 'Landroid/view/ViewGroup;->addView',
 99: 'Landroid/content/Intent;->getStringExtra',
 100: 'CLEAR_APP_CACHE',
 101: 'WRITE_SETTINGS',
 102: 'Ljava/util/Date;-><init>',
 103: 'internal',
 104: 'Ljava/util/concurrent/FutureTask;-><init>',
 105: 'Landroid/widget/Button;-><init>',
 106: 'READ_CONTENT_PROVIDER',
 107: 'Landroid/widget/ImageView;-><init>',
 108: 'Ljava/lang/Math;->min',
 109: 'Landroid/graphics/BitmapFactory;->decodeResource',
 110: 'READ_ATTACHMENT',
 111: 'Lorg/json/JSONObject;->length',
 112: 'Ljava/nio/ByteBuffer;->get',
 113: 'ACCOUNT_MANAGER',
 114: 'MANAGE_ACCOUNTS',
 115: 'Landroid/widget/TextView;->setVisibility',
 116: 'Ljava/lang/Throwable;-><init>',
 117: 'Ljava/lang/String;->toCharArray',
 118: 'Ljava/lang/String;->lastIndexOf',
 119: 'Landroid/os/Bundle;->getBoolean',
 120: 'Ljava/lang/IllegalArgumentException;-><init>',
 121: 'ACCESS_MX_MANAGEMENT_FRAMEWORK_SERVICE',
 122: 'Ljava/util/concurrent/FutureTask;->get',
 123: 'Ljava/lang/Exception;-><init>',
 124: 'ACCESS_CHECKIN_PROPERTIES',
 125: 'GOLDENEYE_SECURITY',
 126: 'Ljava/util/ArrayList;->add',
 127: 'Ljava/io/FileOutputStream;-><init>',
 128: 'Landroid/util/Log;->e',
 129: 'Ljava/lang/ClassLoader;->loadClass',
 130: 'Ljava/io/File;-><init>',
 131: 'Ljava/util/ArrayList;-><init>',
 132: 'USE_CREDENTIALS',
 133: 'Ljava/lang/Character;->codePointAt',
 134: 'Ljava/util/Arrays;->toString',
 135: 'Landroid/os/Parcel;->readString',
 136: 'BLUETOOTH_ADMIN',
 137: 'READ_OWNER_DATA',
 138: 'Ljava/util/HashMap;-><init>',
 139: 'Ljava/io/PrintWriter;->print',
 140: 'Landroid/view/LayoutInflater;->inflate',
 141: 'B04872E0D2604B3993CB7A65CA2ACAFA91A03CE1198A4CD4BE6B9844223E35E9',
 142: 'Landroid/graphics/RectF;-><init>',
 143: 'Ljava/io/ByteArrayOutputStream;->write',
 144: 'ACCESS_COARSE_LOCATION',
 145: 'Landroid/graphics/BitmapFactory;->decodeStream',
 146: 'Landroid/util/Log;->w',
 147: 'Lorg/apache/http/client/HttpClient;->execute',
 148: 'Landroid/util/SparseArray;-><init>',
 149: 'Ljava/math/BigDecimal;-><init>',
 150: 'Landroid/content/Context;->startActivity',
 151: 'Landroid/graphics/Canvas;->save',
 152: 'Ljava/util/HashSet;-><init>',
 153: 'Landroid/content/ContentResolver;->query',
 154: 'Landroid/view/View;->requestFocus',
 155: 'Ljava/util/concurrent/ThreadPoolExecutor;-><init>',
 156: 'Ljava/util/Arrays;->copyOf',
 157: 'Ljava/lang/StringBuffer;-><init>',
 158: 'Ljava/lang/String;->format',
 159: 'Ljava/lang/reflect/Method;->invoke',
 160: 'Ljava/util/GregorianCalendar;-><init>',
 161: 'Landroid/os/Handler;->removeMessages',
 162: 'Landroid/widget/Scroller;->startScroll',
 163: 'Landroid/view/animation/PathInterpolator;-><init>',
 164: 'Ljava/util/TreeSet;-><init>',
 165: 'Landroid/os/Bundle;-><init>',
 166: 'Landroid/util/SparseArray;->get',
 167: 'Landroid/widget/RelativeLayout;-><init>',
 168: 'Landroid/webkit/WebView;-><init>',
 169: 'Landroid/content/res/Resources;->getColor',
 170: 'Ljava/lang/Object;->wait',
 171: 'SMARTCARD',
 172: 'Landroid/graphics/RectF;->set',
 173: 'Landroid/widget/ArrayAdapter;-><init>',
 174: 'CONNECTIVITY_CHANGE',
 175: 'Landroid/view/accessibility/AccessibilityNodeInfo$CollectionItemInfo;->obtain',
 176: 'Ljava/io/File;->isFile',
 177: 'v1',
 178: 'Ljava/lang/Integer;->parseInt',
 179: 'Landroid/view/View;->getLayoutParams',
 180: 'Ljava/math/BigInteger;-><init>',
 181: 'Landroid/content/Context;->registerReceiver',
 182: 'Landroid/graphics/Paint;-><init>',
 183: 'READ_RESULTS',
 184: 'PACKAGE_USAGE_STATS',
 185: 'Landroid/view/View$AccessibilityDelegate;->performAccessibilityAction',
 186: 'Landroid/graphics/Bitmap;->createBitmap',
 187: 'READ_EXTENSION_DATA',
 188: 'Landroid/view/animation/ScaleAnimation;->setInterpolator',
 189: 'Landroid/database/Cursor;->moveToPosition',
 190: 'BIND_ACCESSIBILITY_SERVICE',
 191: 'Landroid/view/View;-><init>',
 192: 'BLUETOOTH',
 193: 'Landroid/view/LayoutInflater;->getFactory',
 194: 'Landroid/content/Context;->getPackageManager',
 195: 'Lorg/json/JSONArray;-><init>',
 196: 'Landroid/graphics/Point;-><init>',
 197: 'WRITE_INTERNAL_STORAGE',
 198: 'Landroid/widget/LinearLayout;-><init>',
 199: 'Ljava/io/File;->delete',
 200: 'WRITE_MEDIA_STORAGE',
 201: 'CALL_AMAZON_DEVICE_INFORMATION_PROVIDER',
 202: 'CHECK_LICENSE',
 203: 'WRITE_EXTERNAL_STORAGE',
 204: 'Landroid/widget/LinearLayout$LayoutParams;-><init>',
 205: 'Landroid/hardware/fingerprint/FingerprintManager$CryptoObject;-><init>',
 206: 'Landroid/widget/PopupWindow;->update',
 207: 'GET_TASKS',
 208: 'Landroid/graphics/drawable/Drawable;->setColorFilter',
 209: 'Ljava/io/InputStreamReader;-><init>',
 210: 'READ_SCHEDULE',
 211: 'BROADCAST_STICKY',
 212: 'Ljava/nio/CharBuffer;->wrap',
 213: 'Ljava/lang/UnsupportedOperationException;-><init>',
 214: 'Landroid/content/Intent;-><init>',
 215: 'Landroid/view/View$MeasureSpec;->getMode',
 216: 'Landroid/view/Gravity;->applyDisplay',
 217: 'WRITE_USE_APP_FEATURE_SURVEY',
 218: 'SET_PREFERRED_APPLICATIONS',
 219: 'Ljava/util/LinkedList;-><init>',
 220: 'signature',
 221: 'Landroid/app/AlertDialog$Builder;-><init>',
 222: 'Landroid/text/Html;->fromHtml',
 223: 'Ljava/util/concurrent/LinkedBlockingQueue;-><init>',
 224: 'Landroid/widget/TextView;->setTextAppearance',
 225: 'Ljava/util/LinkedHashMap;-><init>',
 226: 'SET_WALLPAPER_HINTS',
 227: 'Ljava/util/ArrayList;->remove',
 228: 'READ_CONTACTS',
 229: 'Landroid/os/Handler;->postAtTime',
 230: 'Ljava/io/FileInputStream;->close',
 231: 'Ljava/lang/String;->valueOf',
 232: 'MDM',
 233: 'Ljava/util/Timer;-><init>',
 234: 'Landroid/app/Activity;->startActivityForResult',
 235: 'android',
 236: 'Landroid/widget/FrameLayout;->addView',
 237: 'ANT',
 238: 'SET_ALARM',
 239: 'Landroid/widget/TextView;->setTextColor',
 240: 'Landroid/widget/TextView;->setText',
 241: 'READ_CLOCK',
 242: 'RUN_INSTRUMENTATION',
 243: 'Landroid/graphics/Canvas;->drawBitmap',
 244: 'USE_BIOMETRIC',
 245: 'ACCESS_FINE_LOCATION',
 246: 'READ_SETTINGS',
 247: 'Ljava/lang/NullPointerException;-><init>',
 248: 'Landroid/view/VelocityTracker;->getXVelocity',
 249: 'Landroid/view/View;->setTag',
 250: 'NAVIGATION',
 251: 'Landroid/widget/EdgeEffect;->onPull',
 252: 'Ljava/io/OutputStream;->write',
 253: 'Landroid/app/AlertDialog$Builder;->setTitle',
 254: 'Ljava/io/FileInputStream;-><init>',
 255: 'Landroid/view/animation/AccelerateInterpolator;-><init>'}

In [ ]:
lookUpTable_green = {0: 'FavoritesActivity',
 1: 'NativeBridgeActivity',
 2: 'READ_CALL_LOG',
 3: 'UpdatePasswordActivity',
 4: 'Main2Activity',
 5: 'FBUnityJoinGameGroupActivity',
 6: 'ServiceReceiver',
 7: 'android.intent.action.DOWNLOAD_NOTIFICATION_CLICKED',
 8: 'ForumReplyPostActivity',
 9: 'TriggerReceiver',
 10: 'LocationService',
 11: 'LoadingActivity',
 12: 'PodcastPlayerService',
 13: 'android.intent.action.MAINACTIVITY',
 14: 'WXEntryActivity',
 15: 'OrderDetailsActivity',
 16: 'LOCATION_HARDWARE',
 17: 'com.google.android.c2dm.intent.RETRY',
 18: 'CheckUpdateService',
 19: 'DetailsActivity',
 20: 'PresageActivity',
 21: 'ACCESS_THIRD_PARTY_APP_AUTHORIZATION',
 22: 'com.google.android.c2dm.intent.REGISTRATION',
 23: 'LiveAlbumCameraActivity',
 24: 'SplashScreenActivity',
 25: 'RestoreReceiver',
 26: 'ACCESS_MODULE_API',
 27: 'ForumMessageActivity',
 28: 'umeng',
 29: 'PERMISSION_HISTORY_DATA',
 30: 'ProfileActivity',
 31: 'ImagePagerActivity',
 32: 'SendMessageService',
 33: 'i18n_security',
 34: 'STATUS_BAR',
 35: 'PhoneNumberActionBarActivity',
 36: 'ginlemon.smartlauncher.CLOCKFONTS',
 37: 'FlurryFullscreenTakeoverActivity',
 38: 'com.tencent.mm.plugin.openapi.Intent.ACTION_REFRESH_WXAPP',
 39: 'C2D_MESSAGE',
 40: 'ACCESS_ACCOUNT',
 41: 'ImageCacher',
 42: 'com.phonegap.plugins.barcodescanner.SCAN',
 43: 'SettingActivity',
 44: 'com.fitnessmobileapps.dragonflyyogastudiobrockley',
 45: 'AppInviteDialogActivity',
 46: 'WRITE_CALENDAR',
 47: 'ActionService',
 48: 'com.slideme.sam.manager.inapp.InAppService.BIND',
 49: 'android.intent.action.SEARCH',
 50: 'FuturePaymentInfoActivity',
 51: 'AppEntry',
 52: 'NotificationActivity',
 53: 'EmailActivity',
 54: 'HomeScreenActivity',
 55: 'SearchableActivity',
 56: 'com.google.android.gms.wearable.CAPABILITY_CHANGED',
 57: 'TwitterLoginActivity',
 58: 'BOOT_COMPLETED',
 59: 'READ_CONFIG',
 60: 'com.android.music.metachanged',
 61: 'NewsDetailActivity',
 62: 'AlbumActivity',
 63: 'CAPTURE_AUDIO_OUTPUT',
 64: 'FullScreenActivity',
 65: 'HandshakeRequestService',
 66: 'android.intent.category.SELECTED_ALTERNATIVE',
 67: 'PicksInterstitialActivity',
 68: 'k1_clock_3x3',
 69: 'android.hardware.usb.action.USB_DEVICE_ATTACHED',
 70: 'BBaseDaemonReceiver',
 71: 'REQUEST_INSTALL_PACKAGES',
 72: 'AboutActivity',
 73: 'REQUEST_IGNORE_BATTERY_OPTIMIZATIONS',
 74: 'android.nfc.action.ADAPTER_STATE_CHANGED',
 75: 'ACCESS_SHARED_DATA',
 76: 'PreferenceActivity',
 77: 'ACCESS_COARSE_UPDATES',
 78: 'com.squareup.picasso.PicassoProvider',
 79: 'WebBrowserActivity',
 80: 'A4S_WRITE_DATABASE',
 81: 'Settings',
 82: 'androidx.core.content.FileProvider',
 83: 'cn.jpush.android.intent.NOTIFICATION_RECEIVED',
 84: 'android.appwidget.action.APPWIDGET_CONFIGURE',
 85: 'AudienceNetworkActivity',
 86: 'Search',
 87: 'zendesk.belvedere.BelvedereFileProvider',
 88: 'PushNotificationService',
 89: 'IntroActivity',
 90: 'StoreActivity',
 91: 'StartActivity',
 92: 'notification.action.weather_condition_click',
 93: 'org.zywx.wbpalmstar.widgetone.security',
 94: 'IABBillingService',
 95: 'AppLovinConfirmationActivity',
 96: 'android.intent.category.DEFAULT',
 97: 'CategoryDisplayActivity',
 98: 'com.intent.action.ACTION_STUB_PLUGIN',
 99: 'LOCATION',
 100: 'cn.jpush.android.intent.UNREGISTRATION',
 101: 'org.onepf.oms.billing.BIND',
 102: 'PostIdeaActivity',
 103: 'FacebookActivity',
 104: 'RewardedMraidActivity',
 105: 'AboutAppActivity',
 106: 'tv.ouya.intent.category.GAME',
 107: 'ginlemon.smartlauncher.BUBBLESTYLE',
 108: 'ActivityDynamicLayoutItemActivity',
 109: 'AppSettingsDialogHolderActivity',
 110: 'alarma',
 111: 'MessagingService',
 112: 'AlarmManagerBroadcastReceiver',
 113: 'com.ibuildapp.romanblack.VideoPlugin.PUSH',
 114: 'AppLovinInterstitialActivity',
 115: 'MessageCenterActivity',
 116: 'android.intent.action.LOCALE_CHANGED',
 117: 'UnityDownloaderActivity',
 118: 'com.huawei.android.push.intent.REGISTRATION',
 119: 'GCMRefreshService',
 120: 'UserInfoActivity',
 121: 'AnalyticsService',
 122: 'READ_PHONE_NUMBERS',
 123: 'AGAPAO_INTERNET',
 124: 'TiPreferencesActivity',
 125: 'LoginOrSignupActivity',
 126: 'AroundUsActivity',
 127: 'FirebaseIDService',
 128: 'FuturePaymentConsentActivity',
 129: 'ProductDetailActivity',
 130: 'RemoteControlReceiver',
 131: 'CheckoutActivity',
 132: 'USE_CREDENTIALS',
 133: 'Main',
 134: 'LocationActivity',
 135: 'YouTubePlayerActivity',
 136: 'PurchaseActivity',
 137: 'Home',
 138: 'LoginShim',
 139: 'android.intent.action.DOWNLOAD_COMPLETE',
 140: 'PaymentMethodActivity',
 141: 'FavoriteActivity',
 142: 'UPLOAD_MEDIA',
 143: 'AndromoPreferencesActivity',
 144: 'DynamicLayoutActivity',
 145: 'NOTIFICATION',
 146: 'AlarmHeartBeatReceiver',
 147: 'ActionReceiver',
 148: 'com.rogro.GDE.THEME.1',
 149: 'android.intent.action.DIAL',
 150: 'MetricaService',
 151: 'PayPalTouchActivity',
 152: 'START',
 153: 'DisqusCommentActivity',
 154: 'MediaPlayerActivity',
 155: 'EncodeActivity',
 156: 'com.firebase.jobdispatcher.ACTION_EXECUTE',
 157: 'TimeChangeBroadcastReceiver',
 158: 'aviary.intent.action.ALERT',
 159: 'GCMBroadcastReceiver',
 160: 'SUBSCRIBED_FEEDS_WRITE',
 161: 'TutorialActivity',
 162: 'ImageViewerActivity',
 163: 'com.guidebook.android.PUSH',
 164: 'ReceiveTransitionsIntentService',
 165: 'android.intent.action.SEND',
 166: 'AdColonyAdViewActivity',
 167: 'NotificationsActivity',
 168: 'android.intent.action.ACTION_DOWNLOAD_COMPLETE',
 169: 'ThreeDSecureWebViewActivity',
 170: 'ContactsMapActivity',
 171: 'SMARTCARD',
 172: 'AUTH_SERVICE',
 173: 'HelperActivity',
 174: 'ADMMessageHandler$Receiver',
 175: 'SUPPORT_PERMISSION',
 176: 'com.millennialmedia.internal.utils.MediaContentProvider',
 177: 'v1',
 178: 'android.intent.action.EVENT_REMINDER',
 179: 'PackageAddedReceiver',
 180: 'com.meizu.c2dm.intent.REGISTRATION',
 181: 'android.intent.action.QUICKBOOT_POWERON',
 182: 'ImageGridActivity',
 183: 'common.fuehrerschein.ActivityMenuMain',
 184: 'com.sina.weibo.sdk.action.ACTION_SDK_REQ_ACTIVITY',
 185: 'AppUpdateReceiver',
 186: 'PlayerActivity',
 187: 'CarFinderActivity',
 188: 'CommentListActivity',
 189: 'ACCESS_FINGERPRINT_MANAGER',
 190: 'VungleActivity',
 191: 'READ_CALENDAR',
 192: 'RadioService',
 193: 'VPAIDActivity',
 194: 'UserBehaviorLogService',
 195: 'ParseBroadcastReceiver',
 196: 'com.zendesk.belvedere.BelvedereFileProvider',
 197: 'CouponingDetailsActivity',
 198: 'FREQUENCY_CAP_SECURITY',
 199: 'READ_ONLY',
 200: 'BaseActivity',
 201: 'BootReceiver',
 202: 'ProductListActivity',
 203: 'WRITE_EXTERNAL_STORAGE',
 204: 'aviary.intent.action.CDS_DOWNLOAD_EXTRA_ASSETS',
 205: 'ChatActivity',
 206: 'com.baidu.android.pushservice.action.media.CLICK',
 207: 'ParseFirebaseInstanceIdService',
 208: 'FCM_PLUGIN_ACTIVITY',
 209: 'MraidBrowser',
 210: 'MainActivity',
 211: 'UnityPlayerActivity',
 212: 'changed',
 213: 'android.intent.action.PHONE_STATE',
 214: 'FetchJobService',
 215: 'CartActivity',
 216: 'BootCompletedIntentReceiver',
 217: 'com.google.android.gms.appinvite.ACTION_PREVIEW',
 218: 'VideoPlayActivity',
 219: 'C2DMPushReceiver',
 220: 'VideoFullScreenAdActivity',
 221: 'READ_MESSAGES',
 222: 'finance',
 223: 'READ_PHONE_STATE',
 224: 'com.google.zxing.client.android.ENCODE',
 225: 'ContactUsActivity',
 226: 'MenuActivity',
 227: 'com.androidnative.push.intent.OPEN',
 228: 'Browser',
 229: 'android.media.action.DISPLAY_NOTIFICATION',
 230: 'ChangePasswordActivity',
 231: 'CHANGE_CONFIGURATION',
 232: 'VungleFlexViewActivity',
 233: 'PowerSaveModeBroadcastReceiver',
 234: 'NewsActivity',
 235: 'MessagingActivity',
 236: 'EnableWalletOptimizationReceiver',
 237: 'TestActivity',
 238: 'InterstitialAdActivity',
 239: 'com.google.android.gms.wearable.BIND_LISTENER',
 240: 'ContainerActivity',
 241: 'ReminderManager',
 242: 'REPLACE_WITH_BUNDLE_IDENTIFIER',
 243: 'com.baidu.android.pushservice.action.BIND_SYNC',
 244: 'H5PayActivity',
 245: 'android.intent.action.TIME_SET',
 246: 'android.intent.action.ABOUT',
 247: 'SingleSignInActivity',
 248: 'io.presage.receiver.NetworkChangeReceiver.ONDESTROY',
 249: 'Login',
 250: 'RateAppActivity',
 251: 'com.baidu.android.pushservice.action.METHOD',
 252: 'android.intent.category.BROWSABLE',
 253: 'com.narvii.action.PUSH_START',
 254: 'ScheduleService',
 255: 'FirebasePluginMessagingService'}